In [1]:
# | default_exp preprocess.bigdata.dataloader

In [2]:
# | export

import gc

# from torch.serialization import safe_globals
import os
import pickle
from pathlib import Path
from typing import Optional

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch.utils.data import DataLoader, Dataset

In [3]:
# | export
import logging
import warnings

# Configure logging
logging.basicConfig(level=logging.ERROR)  # Change to DEBUG for more details

logger = logging.getLogger(__name__)

In [4]:
# | export
device = "cuda" if torch.cuda.is_available() else "cpu"
torch._dynamo.config.suppress_errors = True

/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:936: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [5]:
# | export
import gc
import os
import pickle
from pathlib import Path

import h5py
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import ray
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm


class TSPreprocessor:
    def __init__(
        self,
        df,
        input_size,
        horizon,
        target_col="y_scaled",
        train_split=0.7,
        val_split=0.15,
        split_type="horizontal",
        step_size=1,
        cache_dir=".",
        use_cache=True,
        experiment_name="default_experiment",
        num_workers=4,
        chunk_size=1000,
        adaptive_step=False,
        downsample_factor=1,
        large_dataset=True,
        save_by_window_count=False,
        max_windows_per_file=2000,
        batch_size=200,
    ):
        logger.info("Initializing TSPreprocessor")
        self.df = df
        self.target_col = target_col if target_col in df.columns else "y"
        self.input_size = input_size
        self.horizon = horizon
        self.train_split = train_split
        self.val_split = val_split
        self.split_type = split_type
        self.step_size = step_size
        self.cache_dir = Path(cache_dir) / experiment_name
        self.cache_dir.mkdir(exist_ok=True, parents=True)
        self.use_cache = use_cache
        self.num_workers = min(num_workers, os.cpu_count() or 4)
        self.experiment_name = experiment_name
        self.chunk_size = chunk_size
        self.adaptive_step = adaptive_step
        self.downsample_factor = downsample_factor
        self.large_dataset = large_dataset
        self.save_by_window_count = save_by_window_count
        self.max_windows_per_file = max_windows_per_file
        self.batch_size = batch_size

        if not 0 < train_split + val_split <= 1:
            raise ValueError("train_split + val_split must be between 0 and 1")
        if step_size < 1:
            raise ValueError("step_size must be >= 1")
        if downsample_factor < 1:
            raise ValueError("downsample_factor must be >= 1")
        if max_windows_per_file < 1:
            raise ValueError("max_windows_per_file must be >= 1")
        if batch_size < 1:
            raise ValueError("batch_size must be >= 1")

        # Initialize Ray
        if not ray.is_initialized():
            ray.init(num_cpus=self.num_workers, ignore_reinit_error=True)

        self.train_windows, self.val_windows, self.test_windows = self._process_data()

        del self.df
        gc.collect()

    def _generate_windows_in_memory(self, series):
        series_len = len(series)
        if series_len < self.input_size + self.horizon:
            return []

        max_idx = series_len - self.input_size - self.horizon + 1
        if max_idx <= 0:
            return []

        window_starts = np.arange(0, max_idx, self.step_size, dtype=np.int32)
        window_ends = window_starts + self.input_size
        horizon_ends = window_ends + self.horizon

        valid_windows = horizon_ends <= series_len
        window_starts = window_starts[valid_windows]
        window_ends = window_ends[valid_windows]
        horizon_ends = horizon_ends[valid_windows]

        x_windows = np.lib.stride_tricks.sliding_window_view(series, window_shape=self.input_size)[
            window_starts
        ]
        y_windows = np.stack([series[we:he] for we, he in zip(window_ends, horizon_ends)])

        return list(zip(x_windows, y_windows))

    @staticmethod
    def _generate_windows_optimized(
        series,
        unique_id,
        split,
        window_buffer=None,
        chunk_size=None,
        input_size=None,
        horizon=None,
        step_size=None,
        adaptive_step=False,
        cache_dir=None,
    ):
        series_len = len(series)
        if series_len < input_size + horizon:
            return window_buffer

        max_idx = series_len - input_size - horizon + 1
        if max_idx <= 0:
            return window_buffer

        step_size = step_size
        if adaptive_step:
            step_size = max(1, series_len // 1000)

        chunk_size = chunk_size or max(100, min(1000, series_len // 10))

        if window_buffer is not None:
            if not isinstance(window_buffer, list):
                window_buffer = list(window_buffer)
            for start in range(0, max_idx, chunk_size * step_size):
                end = min(start + chunk_size * step_size, max_idx)
                window_starts = np.arange(start, end, step_size, dtype=np.int32)
                window_ends = window_starts + input_size
                horizon_ends = window_ends + horizon

                valid_windows = horizon_ends <= series_len
                window_starts = window_starts[valid_windows]
                window_ends = window_ends[valid_windows]
                horizon_ends = horizon_ends[valid_windows]

                x_windows = np.lib.stride_tricks.sliding_window_view(
                    series, window_shape=input_size
                )[window_starts]
                y_windows = np.stack([series[we:he] for we, he in zip(window_ends, horizon_ends)])

                window_buffer.extend(zip(x_windows, y_windows))

                del x_windows, y_windows
                gc.collect()

            return window_buffer
        else:
            h5_file = Path(cache_dir) / f"{unique_id}_{split}.h5"
            with h5py.File(h5_file, "a") as f:
                x_dset = (
                    f["x"]
                    if "x" in f
                    else f.create_dataset(
                        "x",
                        shape=(0, input_size),
                        maxshape=(None, input_size),
                        dtype=np.float32,
                        compression="lzf",
                    )
                )
                y_dset = (
                    f["y"]
                    if "y" in f
                    else f.create_dataset(
                        "y",
                        shape=(0, horizon),
                        maxshape=(None, horizon),
                        dtype=np.float32,
                        compression="lzf",
                    )
                )

                for start in range(0, max_idx, chunk_size * step_size):
                    end = min(start + chunk_size * step_size, max_idx)
                    window_starts = np.arange(start, end, step_size, dtype=np.int32)
                    window_ends = window_starts + input_size
                    horizon_ends = window_ends + horizon

                    valid_windows = horizon_ends <= series_len
                    window_starts = window_starts[valid_windows]
                    window_ends = window_ends[valid_windows]
                    horizon_ends = horizon_ends[valid_windows]

                    x_windows = np.lib.stride_tricks.sliding_window_view(
                        series, window_shape=input_size
                    )[window_starts]
                    y_windows = np.stack(
                        [series[we:he] for we, he in zip(window_ends, horizon_ends)]
                    )

                    x_dset.resize(x_dset.shape[0] + len(x_windows), axis=0)
                    y_dset.resize(y_dset.shape[0] + len(y_windows), axis=0)
                    x_dset[-len(x_windows) :] = x_windows
                    y_dset[-len(y_windows) :] = y_windows

                    del x_windows, y_windows
                    gc.collect()

    @staticmethod
    @ray.remote
    def _save_buffered_windows(window_buffer, file_idx, split, cache_dir, input_size, horizon):
        if not window_buffer:
            return file_idx

        h5_file = Path(cache_dir) / f"windows_{file_idx}_{split}.h5"
        with h5py.File(h5_file, "a") as f:
            x_dset = (
                f["x"]
                if "x" in f
                else f.create_dataset(
                    "x",
                    shape=(0, input_size),
                    maxshape=(None, input_size),
                    dtype=np.float32,
                    compression="lzf",
                )
            )
            y_dset = (
                f["y"]
                if "y" in f
                else f.create_dataset(
                    "y",
                    shape=(0, horizon),
                    maxshape=(None, horizon),
                    dtype=np.float32,
                    compression="lzf",
                )
            )

            x_windows, y_windows = zip(*window_buffer)
            x_windows = np.stack(x_windows)
            y_windows = np.stack(y_windows)

            x_dset.resize(x_dset.shape[0] + len(x_windows), axis=0)
            y_dset.resize(y_dset.shape[0] + len(y_windows), axis=0)
            x_dset[-len(x_windows) :] = x_windows
            y_dset[-len(y_windows) :] = y_windows

        del window_buffer[:]
        gc.collect()
        return file_idx + 1

    def _process_one_series_in_memory(self, unique_id_group):
        unique_id, group = unique_id_group
        series = group[self.target_col].values.astype(np.float32)

        windows = self._generate_windows_in_memory(series)
        del series
        gc.collect()

        if not windows:
            logger.warning(f"[{unique_id}] Series too short to generate any windows")
            return [], [], [], unique_id

        if self.split_type == "horizontal":
            num_windows = len(windows)
            train_end = int(num_windows * self.train_split)
            val_end = train_end + int(num_windows * self.val_split)
            return (windows[:train_end], windows[train_end:val_end], windows[val_end:], unique_id)
        else:
            return windows, [], [], unique_id

    @staticmethod
    @ray.remote(num_cpus=1)  # Allocate 1 CPU per task for full utilization
    def _process_batch_optimized(
        unique_id_groups,
        window_buffers,
        file_indices,
        cache_dir,
        input_size,
        horizon,
        step_size,
        adaptive_step,
        downsample_factor,
        chunk_size,
        max_windows_per_file,
        split_type,
        train_split,
        val_split,
        target_col,
    ):
        results = []
        for unique_id_group in unique_id_groups:
            unique_id, group = unique_id_group
            series = group[target_col].values.astype(np.float32)

            if len(series) < input_size + horizon:
                continue  # Skip short series

            if downsample_factor > 1:
                series = series[::downsample_factor]

            step_size = step_size
            if adaptive_step:
                step_size = max(1, len(series) // 1000)

            if split_type == "horizontal":
                series_len = len(series)
                num_windows = max(0, (series_len - input_size - horizon + 1) // step_size)
                train_end = int(num_windows * train_split)
                val_end = train_end + int(num_windows * val_split)

                result = []
                if window_buffers is not None:
                    for split, start, end in [
                        ("train", 0, train_end * step_size),
                        ("val", train_end * step_size, val_end * step_size),
                        ("test", val_end * step_size, series_len),
                    ]:
                        buffer = TSPreprocessor._generate_windows_optimized(
                            series[start:end],
                            unique_id,
                            split,
                            window_buffers.get(split),
                            chunk_size,
                            input_size,
                            horizon,
                            step_size,
                            adaptive_step,
                            cache_dir,
                        )
                        if buffer:
                            window_buffers[split] = buffer
                        result.append((None, unique_id, split))
                    results.append((result, window_buffers, file_indices))
                else:
                    for split, start, end in [
                        ("train", 0, train_end * step_size),
                        ("val", train_end * step_size, val_end * step_size),
                        ("test", val_end * step_size, series_len),
                    ]:
                        TSPreprocessor._generate_windows_optimized(
                            series[start:end],
                            unique_id,
                            split,
                            None,
                            chunk_size,
                            input_size,
                            horizon,
                            step_size,
                            adaptive_step,
                            cache_dir,
                        )
                        result.append(
                            (Path(cache_dir) / f"{unique_id}_{split}.h5", unique_id, split)
                        )
                    results.append((result, window_buffers, file_indices))
            else:
                if window_buffers is not None:
                    buffer = TSPreprocessor._generate_windows_optimized(
                        series,
                        unique_id,
                        "all",
                        window_buffers.get("all"),
                        chunk_size,
                        input_size,
                        horizon,
                        step_size,
                        adaptive_step,
                        cache_dir,
                    )
                    if buffer:
                        window_buffers["all"] = buffer
                    results.append(([(None, unique_id, "all")], window_buffers, file_indices))
                else:
                    TSPreprocessor._generate_windows_optimized(
                        series,
                        unique_id,
                        "all",
                        None,
                        chunk_size,
                        input_size,
                        horizon,
                        step_size,
                        adaptive_step,
                        cache_dir,
                    )
                    results.append(
                        (
                            [(Path(cache_dir) / f"{unique_id}_all.h5", unique_id, "all")],
                            window_buffers,
                            file_indices,
                        )
                    )
        return results

    def _process_data(self):
        logger.info("Processing data for all unique_ids")
        cache_file = self.cache_dir / "preprocessed_windows.pkl"

        if self.use_cache and cache_file.exists():
            logger.info("Loading preprocessed windows from cache")
            with open(cache_file, "rb") as f:
                data = pickle.load(f)
            return data["train_windows"], data["val_windows"], data["test_windows"]

        grouped = list(self.df.groupby("unique_id"))
        train_windows, val_windows, test_windows = [], [], []

        if self.split_type == "vertical":
            logger.info("Applying vertical split")
            unique_ids = list(self.df["unique_id"].unique())
            np.random.shuffle(unique_ids)
            total_series = len(unique_ids)
            train_end = int(total_series * self.train_split)
            val_end = train_end + int(total_series * self.val_split)

            train_ids = unique_ids[:train_end]
            val_ids = unique_ids[train_end:val_end]
            test_ids = unique_ids[val_end:]

        if self.large_dataset:
            window_buffers = {"train": [], "val": [], "test": [], "all": []}
            file_indices = {"train": 0, "val": 0, "test": 0, "all": 0}

            # Batch unique IDs to reduce task overhead
            batches = [
                grouped[i : i + self.batch_size] for i in range(0, len(grouped), self.batch_size)
            ]

            # Submit Ray tasks for processing batches
            futures = [
                TSPreprocessor._process_batch_optimized.remote(
                    batch,
                    window_buffers,
                    file_indices,
                    self.cache_dir,
                    self.input_size,
                    self.horizon,
                    self.step_size,
                    self.adaptive_step,
                    self.downsample_factor,
                    self.chunk_size,
                    self.max_windows_per_file,
                    self.split_type,
                    self.train_split,
                    self.val_split,
                    self.target_col,
                )
                for batch in batches
            ]

            # Collect results with tqdm progress bar
            pbar = tqdm(total=len(futures), desc="Processing batches", dynamic_ncols=True)
            results = []
            while futures:
                done, futures = ray.wait(futures, num_returns=min(len(futures), 1))
                batch_results = ray.get(done)[0]  # Unpack batch results
                results.extend(batch_results)
                pbar.update(len(done))
            pbar.close()

            if self.save_by_window_count:
                futures = []
                for split in window_buffers:
                    buffer = window_buffers[split]
                    file_idx = file_indices[split]
                    while buffer:
                        futures.append(
                            TSPreprocessor._save_buffered_windows.remote(
                                buffer[: self.max_windows_per_file],
                                file_idx,
                                split,
                                self.cache_dir,
                                self.input_size,
                                self.horizon,
                            )
                        )
                        train_windows.append(
                            (self.cache_dir / f"windows_{file_idx}_{split}.h5", None)
                        )
                        buffer = buffer[self.max_windows_per_file :]
                        file_idx += 1
                    file_indices[split] = file_idx

                # Save windows with tqdm progress bar
                pbar = tqdm(total=len(futures), desc="Saving windows", dynamic_ncols=True)
                while futures:
                    done, futures = ray.wait(futures, num_returns=min(len(futures), 1))
                    for file_idx in ray.get(done):
                        file_indices[split] = max(file_indices[split], file_idx)
                    pbar.update(len(done))
                pbar.close()

            for result, _, _ in results:
                for h5_file, unique_id, split in result:
                    if h5_file:
                        if self.split_type == "horizontal":
                            if split == "train":
                                train_windows.append((h5_file, unique_id))
                            elif split == "val":
                                val_windows.append((h5_file, unique_id))
                            elif split == "test":
                                test_windows.append((h5_file, unique_id))
                        else:
                            if unique_id in train_ids:
                                train_windows.append((h5_file, unique_id))
                            elif unique_id in val_ids:
                                val_windows.append((h5_file, unique_id))
                            elif unique_id in test_ids:
                                test_windows.append((h5_file, unique_id))
        else:
            from concurrent.futures import ProcessPoolExecutor

            with ProcessPoolExecutor(max_workers=self.num_workers) as executor:
                results = list(
                    tqdm(
                        executor.map(self._process_one_series_in_memory, grouped),
                        total=len(grouped),
                        desc="Processing series",
                    )
                )

            for train_w, val_w, test_w, unique_id in results:
                if self.split_type == "horizontal":
                    train_windows.extend(train_w)
                    val_windows.extend(val_w)
                    test_windows.extend(test_w)
                else:
                    if unique_id in train_ids:
                        train_windows.extend(train_w)
                    elif unique_id in val_ids:
                        val_windows.extend(train_w)
                    elif unique_id in test_ids:
                        test_windows.extend(train_w)

        train_windows = np.array(train_windows, dtype=object)
        val_windows = np.array(val_windows, dtype=object)
        test_windows = np.array(test_windows, dtype=object)

        if self.use_cache:
            logger.info("Saving preprocessed windows metadata to cache")
            with open(cache_file, "wb") as f:
                pickle.dump(
                    {
                        "train_windows": train_windows,
                        "val_windows": val_windows,
                        "test_windows": test_windows,
                    },
                    f,
                )

        logger.info(
            f"Train windows: {len(train_windows)}, Val windows: {len(val_windows)}, Test windows: {len(test_windows)}"
        )
        return train_windows, val_windows, test_windows


class UnivariateTSDataset(Dataset):
    def __init__(self, windows, device: str = None, large_dataset: bool = True):
        logger.info("Initializing UnivariateTSDataset")
        self.windows = windows
        self.device = device
        self.large_dataset = large_dataset
        if large_dataset:
            self.h5_files = {}
        else:
            self.x = np.stack([w[0] for w in windows], axis=0).astype(np.float32)
            self.y = np.stack([w[1] for w in windows], axis=0).astype(np.float32)
            if device:
                logger.info("Preloading tensors to device: %s", device)
                self.x_tensor = torch.from_numpy(self.x).to(device)
                self.y_tensor = torch.from_numpy(self.y).to(device)
            else:
                self.x_tensor = self.y_tensor = None

    def __len__(self):
        if self.large_dataset:
            total_len = 0
            for h5_file, _ in self.windows:
                if h5_file not in self.h5_files:
                    self.h5_files[h5_file] = h5py.File(h5_file, "r")
                total_len += self.h5_files[h5_file]["x"].shape[0]
            return total_len
        return len(self.x)

    def __getitem__(self, idx):
        if self.large_dataset:
            current_idx = 0
            for h5_file, _ in self.windows:
                if h5_file not in self.h5_files:
                    self.h5_files[h5_file] = h5py.File(h5_file, "r")
                num_windows = self.h5_files[h5_file]["x"].shape[0]
                if current_idx + num_windows > idx:
                    local_idx = idx - current_idx
                    x = self.h5_files[h5_file]["x"][local_idx]
                    y = self.h5_files[h5_file]["y"][local_idx]
                    if self.device:
                        return torch.from_numpy(x).to(self.device), torch.from_numpy(y).to(
                            self.device
                        )
                    return torch.from_numpy(x), torch.from_numpy(y)
                current_idx += num_windows
            raise IndexError("Index out of range")
        else:
            if self.device:
                return self.x_tensor[idx], self.y_tensor[idx]
            return torch.from_numpy(self.x[idx]), torch.from_numpy(self.y[idx])

    def __del__(self):
        if self.large_dataset:
            for h5_file in self.h5_files.values():
                h5_file.close()


class UnivariateTSDataModule(pl.LightningDataModule):
    def __init__(
        self,
        preprocessor: TSPreprocessor,
        batch_size=32,
        num_workers=8,
        pin_memory=True,
        prefetch_factor=2,
        persistent_workers=True,
        gpu_preload=False,
        experiment_name="default_experiment",
    ):
        logger.info("Initializing UnivariateTSDataModule")
        super().__init__()
        self.save_hyperparameters(ignore=["preprocessor"])
        self.preprocessor = preprocessor
        self.batch_size = batch_size
        self.num_workers = min(num_workers, torch.get_num_threads())
        self.pin_memory = pin_memory and not gpu_preload
        self.prefetch_factor = prefetch_factor
        self.persistent_workers = persistent_workers
        self.gpu_preload = gpu_preload
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.experiment_name = experiment_name

    def setup(self, stage: str = None):
        logger.info(f"Setting up datamodule for stage: {stage}")
        if stage in ("fit", None):
            self.train_dataset = UnivariateTSDataset(
                self.preprocessor.train_windows,
                device=self.device if self.gpu_preload else None,
                large_dataset=self.preprocessor.large_dataset,
            )
            self.val_dataset = UnivariateTSDataset(
                self.preprocessor.val_windows,
                device=self.device if self.gpu_preload else None,
                large_dataset=self.preprocessor.large_dataset,
            )
        if stage in ("validate", None):
            self.val_dataset = UnivariateTSDataset(
                self.preprocessor.val_windows,
                device=self.device if self.gpu_preload else None,
                large_dataset=self.preprocessor.large_dataset,
            )
        if stage in ("test", None):
            self.test_dataset = UnivariateTSDataset(
                self.preprocessor.test_windows,
                device=self.device if self.gpu_preload else None,
                large_dataset=self.preprocessor.large_dataset,
            )

    def _create_dataloader(self, dataset, shuffle=False):
        logger.info("Creating dataloader")
        return DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=shuffle,
            num_workers=self.num_workers,
            pin_memory=self.pin_memory,
            prefetch_factor=self.prefetch_factor,
            persistent_workers=self.persistent_workers,
            drop_last=shuffle,
        )

    def train_dataloader(self):
        logger.info("Getting train dataloader")
        return self._create_dataloader(self.train_dataset, shuffle=True)

    def val_dataloader(self):
        logger.info("Getting val dataloader")
        return self._create_dataloader(self.val_dataset)

    def test_dataloader(self):
        logger.info("Getting test dataloader")
        return self._create_dataloader(self.test_dataset)

In [ ]:
%%time
# Example usage
## Data loading and preprocessing df.columns - unique_id,ds,y
from datasetsforecast.m5 import M5

df = M5().load("../data")[0]  # , group="Monthly"
df.sort_values(["unique_id", "ds"], inplace=True)


horizon = 180  # <-- FORECAST HORIZON
input_size = horizon * 6

# from ts.preprocess.dataloader import TSPreprocessor, UnivariateTSDataModule
import os

batch_size = 128
num_workers = os.cpu_count() - 6
step_size = 1

# Initialize preprocessor
preprocessor = TSPreprocessor(
    df=df,
    input_size=input_size,
    horizon=horizon,
    target_col="y_scaled",  # Default to y_scaled, falls back to y
    train_split=0.7,
    val_split=0.15,
    split_type="vertical",
    step_size=step_size,
    cache_dir=".",
    use_cache=True,
    experiment_name="m5_experiment",
    num_workers=num_workers,
    chunk_size=1000,
    adaptive_step=False,
    downsample_factor=1,
    large_dataset=True,  # Set to False for original in-memory approach
    save_by_window_count=True,  # Set to True to save by window count
    max_windows_per_file=5000,  # Number of windows per HDF5 file
)

# Initialize DataModule
ds = UnivariateTSDataModule(
    preprocessor=preprocessor,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True,
    prefetch_factor=2,
    persistent_workers=True,
    gpu_preload=False,
    experiment_name="m5_experiment",
)

2025-05-04 16:24:15,272	INFO worker.py:1852 -- Started a local Ray instance.
/tmp/ipykernel_205704/1914704754.py:304: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = list(self.df.groupby("unique_id"))


Processing batches:   0%|                                                                                     …

(raylet) Spilled 6059 MiB, 137 objects, write throughput 777 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.
(raylet) Spilled 6553 MiB, 144 objects, write throughput 706 MiB/s.
(raylet) [2025-05-04 16:25:15,257 E 206186 206186] (raylet) node_manager.cc:3313: 6 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 90f2778e5190063d902fb7fd1d2a566fc5585bc4c0e4efecee09a2b5, IP: 192.168.0.112) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 192.168.0.112`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker ki

In [10]:
%%time
ds.setup()
for x, y in ds.train_dataloader():
    break

NameError: name 'ds' is not defined